<h1 align= "center"> <strong> Pipeline Configurations </strong> </h1>
<center>---------------------------- </center>

# Table of Contents
* [1. Writing JSON to a file in python](#1.-Writing-JSON-to-a-file-in-python)
* [2. Reading JSON from a file using python](#2.-Reading-JSON-from-a-file-using-python)

## 1. Writing JSON to a file in python

In [30]:
#define the project details in here
project_name = "your_projectname"
job_name='inf'
BUName = "your_BUName"
env = 'dev'
#define your tags in here
ProjectName = "ProjectName"
DepartmentName = "DepartmentName"
OwnerName = "OwnerName"
#Once again please add your input datasets name here
input1 = "your_first_input_dataset.csv"
input2 = "your_second_input_dataset.csv"
input3 = "your_third_input_dataset.csv"

In [31]:
import json
import pytz
from datetime import datetime
import boto3
srilanka_tz = pytz.timezone('Asia/Colombo')
s3 = boto3.client('s3')
date_folder = datetime.now(srilanka_tz).strftime("%Y-%m-%d")
  
# Data to be written
dictionary ={
    "pipeline": {
        "pipeline_config" : {
            "model_package_group_name": f"{BUName}{project_name}-ModelPackageGroup",
            "pipeline_name": f"{BUName}-{project_name}-{job_name}-pipeline-",
            "base_job_prefix": f"{BUName}-{project_name}-{job_name}",
            "env": f"{env}",
            "project_name": f"{project_name}",
            "BUName": f"{BUName}",
        },
        "generic_tags": {
            "ProjectName": f"{ProjectName}",
            "DepartmentName": f"{DepartmentName}",
            "UsecaseName": f"{project_name}",
            "ResourceName": "sagemaker_notebook",
            "OwnerName": f"{OwnerName}",
            "BUName": f"{BUName}"
        }
    },
    "preprocessing": {
        "estimator_config": {
            "ecr_repository": f"{BUName}-{project_name}-preprocessing-image",
            "instance_count": 1,
            "instance_type": "ml.m5.large"
        },
        "inputs": {
            "code" : f"{BUName}_{project_name}_{job_name}_preprocessing/preprocessing.py",
            "input1": f"s3://dlk-cloud-tier-10-preprocessed-ml-dev/{project_name}/{job_name}/sample_input1_dataset/{input1}",
            "input2": f"s3://dlk-cloud-tier-10-preprocessed-ml-dev/{project_name}/{job_name}/sample_input2_dataset/{input2}",
            "input3": f"s3://dlk-cloud-tier-10-preprocessed-ml-dev/{project_name}/{job_name}/sample_input3_dataset/{input3}"
        },
    },
    "create_model": {
        "estimator_config": {
            "instance_count": 2,
            "instance_type": "ml.m5.xlarge",
        },
    },
    "inference": {
        "estimator_config": {
            "ecr_repository": f"{BUName}-{project_name}-inf-inferencedt-image",
            "instance_count": 2,
            "instance_type": "ml.m5.2xlarge",
            "content_type": "text/csv"
        }
    },
    "postpreprocessing": {
        "inputs": {
            "code" : f"{BUName}_{project_name}_{job_name}_postprocessing/postprocessing.py"
        },

}
}

In [32]:
import json
import boto3    
s3 = boto3.resource('s3')
s3object = s3.Object('dlk-cloud-tier-8-code-ml-dev', f'config_files/pipeline_{job_name}_config/{BUName}_{project_name}/{date_folder}/pipeline_{job_name}_config.json')

s3object.put(
    Body=(bytes(json.dumps(dictionary).encode('UTF-8')))
)

{'ResponseMetadata': {'RequestId': '1ZJGA9TET7BJR5H1',
  'HostId': 'mw62JcnpslDXLIMrNykQNowyuaZIR0PR94TD7AuqlTD+Jn6DAWv9lfStSd7QIcdE3x6ppUyZcKI=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'mw62JcnpslDXLIMrNykQNowyuaZIR0PR94TD7AuqlTD+Jn6DAWv9lfStSd7QIcdE3x6ppUyZcKI=',
   'x-amz-request-id': '1ZJGA9TET7BJR5H1',
   'date': 'Mon, 14 Mar 2022 11:12:08 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"27d64176dcb6f77208bca1c375dfaa0f"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"27d64176dcb6f77208bca1c375dfaa0f"',
 'ServerSideEncryption': 'AES256'}

## 2. Reading JSON from a file using python

In [33]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

In [34]:
env = 'dev'

In [35]:
import json
import boto3
from sagemaker.workflow.parameters import (
        ParameterInteger,
        ParameterString,
    )


def latest_config():
        
        '''
        this will pick the latest trained config file
        '''

        s3_path = f"s3://dlk-cloud-tier-8-code-ml-{env}/"
        config_bucket = f"dlk-cloud-tier-8-code-ml-{env}"
        s3 = boto3.client('s3')
        all_objects = s3.list_objects(Bucket = f"dlk-cloud-tier-8-code-ml-{env}", Prefix=f"config_files/pipeline_{job_name}_config/{project_name}/")
        model_folder=all_objects['Contents'][-1]['Key']
        #model_path=s3_path+model_folder

        return config_bucket, model_folder

config_s3_bucket = latest_config()[0]
model_s3_path = latest_config()[1]


def getJsonData(bucket_name,key_name):
        '''
        this will pick the json config file from s3 bucket
        '''

        print("[LOG]", bucket_name,'---------')
        print("[LOG]", key_name,'--------------')

        s3_client = boto3.client('s3')
        csv_obj = s3_client.get_object(Bucket=bucket_name, Key=key_name)

        body = csv_obj['Body']

        json_string = body.read().decode('utf-8')
        json_content = json.loads(json_string)

        return json_content


config = getJsonData(config_s3_bucket,model_s3_path)

[LOG] dlk-cloud-tier-8-code-ml-dev ---------
[LOG] config_files/pipeline_inf_config/mobile_prepaidchurn/2022-03-14/pipeline_inf_config.json --------------


### Experiments

In [48]:
config['pipeline']['pipeline_config']

{'model_package_group_name': 'MobilePrepaidChurn-ModelPackageGroup',
 'pipeline_name': 'mobile-prepaidchurn-inf-pipeline-',
 'base_job_prefix': 'mobile-prepaidchurn-inf',
 'env': 'dev',
 'project_name': 'prepaidchurn',
 'BUName': 'mobile'}

In [45]:
"120582440665.dkr.ecr.ap-southeast-1.amazonaws.com/mobile-prepaidchurn-inf-inferencexg-image:latest"

'120582440665.dkr.ecr.ap-southeast-1.amazonaws.com/mobile-prepaidchurn-inf-inferencexg-image:latest'

In [49]:
BU = config['pipeline']['pipeline_config']['BUName']
prj_name = config['pipeline']['pipeline_config']['project_name']

In [46]:
f"120582440665.dkr.ecr.ap-southeast-1.amazonaws.com/{BU}-{prj_name}-inf-inferencexg-image:latest"

'120582440665.dkr.ecr.ap-southeast-1.amazonaws.com/mobile-prepaidchurn-inf-inferencexg-image:latest'

In [43]:
BU = config['pipeline']['pipeline_config']['BUName']
prj_name = config['pipeline']['pipeline_config']['project_name']

f"120582440665.dkr.ecr.ap-southeast-1.amazonaws.com/{BU}-{prj_name}-inf-inferencedt-image:latest"

In [44]:
f"120582440665.dkr.ecr.ap-southeast-1.amazonaws.com/{BU}-{prj_name}-inf-inferencedt-image:latest"

'120582440665.dkr.ecr.ap-southeast-1.amazonaws.com/mobile-prepaidchurn-inf-inferencedt-image:latest'

In [42]:
config['pipeline']['pipeline_config']['model_package_group_name']

'MobilePrepaidChurn-ModelPackageGroup'

In [13]:
config['pipeline']['pipeline_config']['pipeline_name']

'SalesStockPred-pipeline-'

In [14]:
config['pipeline']['pipeline_config']['base_job_prefix']

'SalesStockPred'

In [15]:
config['pipeline']['pipeline_config']['env']

'dev'

In [16]:
config['pipeline']['pipeline_config']['project_name']

'sales_stockpred'

In [18]:
config['pipeline']['inputs']['input1']

's3://dlk-cloud-tier-10-preprocessed-ml-dev/sales_stockpred/temp_data/gc_retailers_3yr/temp/'

In [19]:
config['pipeline']['inputs']['input2']

's3://dlk-cloud-tier-10-preprocessed-ml-dev/sales_stockpred/temp_data/gc_retailers_3yr/gc_retailers_2022_JAN10.csv'

In [24]:
config['pipeline']['generic_tags']['UsecaseName']

'sales-stockpred'

In [28]:
config['preprocessing']['estimator_config']

{'ecr_repository': 'sales-stockpred-preprocessing-image',
 'instance_count': 1,
 'instance_type': 'ml.m5.large'}

In [33]:
config['train_model']['estimator_config']['instance_count']

2